# Transmission

Model energy or material transport between locations with losses.

This notebook covers:

- **Transmission component**: Connecting sites with pipelines, cables, or conveyors
- **Transmission losses**: Relative losses (proportional) and absolute losses (fixed)
- **Bidirectional flow**: Two-way transmission with flow direction constraints
- **Capacity optimization**: Sizing transmission infrastructure

## Setup

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import xarray as xr

import flixopt as fx

fx.CONFIG.notebook()

flixopt.config.CONFIG

## The Problem: Connecting Two Sites

Consider a district heating network with two sites:

- **Site A**: Has a large gas boiler (cheap production)
- **Site B**: Has a smaller electric boiler (expensive, but flexible)

A district heating pipe connects both sites. The question: How should heat flow between sites to minimize total costs?

### Transmission Characteristics

| Parameter | Value | Description |
|-----------|-------|-------------|
| Relative losses | 5% | Heat loss proportional to flow (pipe heat loss) |
| Capacity | 200 kW | Maximum transmission rate |
| Bidirectional | Yes | Heat can flow A→B or B→A |

## Define Time Series Data

In [2]:
# One week simulation
timesteps = pd.date_range('2024-01-22', periods=168, freq='h')
hours = np.arange(168)
hour_of_day = hours % 24

# Site A: Industrial facility with steady demand
demand_a_base = 150
demand_a_variation = 30 * np.sin(hour_of_day * np.pi / 12)  # Day/night cycle
demand_a = demand_a_base + demand_a_variation

# Site B: Office building with peak during work hours
demand_b = np.where(
    (hour_of_day >= 8) & (hour_of_day <= 18),
    180,  # Daytime: 180 kW
    80,  # Nighttime: 80 kW
)
# Add weekly pattern (lower on weekends)
day_of_week = (hours // 24) % 7
demand_b = np.where(day_of_week >= 5, demand_b * 0.6, demand_b)  # Weekend reduction

In [3]:
# Visualize demand profiles
fig = px.line(
    x=timesteps.tolist() * 2,
    y=np.concatenate([demand_a, demand_b]),
    color=['Site A (Industrial)'] * 168 + ['Site B (Office)'] * 168,
    title='Heat Demand at Both Sites',
    labels={'x': 'Time', 'y': 'Heat Demand [kW]', 'color': 'Site'},
)
fig

## Example 1: Unidirectional Transmission

Start with a simple case: heat flows only from Site A to Site B.

In [4]:
fs_unidirectional = fx.FlowSystem(timesteps)
fs_unidirectional.add_carriers(
    fx.Carrier('gas', '#3498db', 'kW'),
    fx.Carrier('electricity', '#f1c40f', 'kW'),
    fx.Carrier('heat', '#e74c3c', 'kW'),
)
fs_unidirectional.add_elements(
    # === Buses (one per site) ===
    fx.Bus('Heat_A', carrier='heat'),  # Site A heat network
    fx.Bus('Heat_B', carrier='heat'),  # Site B heat network
    fx.Bus('Gas', carrier='gas'),  # Gas supply network
    fx.Bus('Electricity', carrier='electricity'),  # Electricity grid
    # === Effect ===
    fx.Effect('costs', '€', 'Operating Costs', is_standard=True, is_objective=True),
    # === External supplies ===
    fx.Source('GasSupply', outputs=[fx.Flow('Gas', bus='Gas', size=1000, effects_per_flow_hour=0.06)]),
    fx.Source('ElecGrid', outputs=[fx.Flow('Elec', bus='Electricity', size=500, effects_per_flow_hour=0.25)]),
    # === Site A: Large gas boiler (cheap) ===
    fx.LinearConverter(
        'GasBoiler_A',
        inputs=[fx.Flow('Gas', bus='Gas', size=500)],
        outputs=[fx.Flow('Heat', bus='Heat_A', size=400)],
        conversion_factors=[{'Gas': 1, 'Heat': 0.92}],  # 92% efficiency
    ),
    # === Site B: Small electric boiler (expensive but flexible) ===
    fx.LinearConverter(
        'ElecBoiler_B',
        inputs=[fx.Flow('Elec', bus='Electricity', size=250)],
        outputs=[fx.Flow('Heat', bus='Heat_B', size=250)],
        conversion_factors=[{'Elec': 1, 'Heat': 0.99}],  # 99% efficiency
    ),
    # === Transmission: A → B (unidirectional) ===
    fx.Transmission(
        'Pipe_A_to_B',
        in1=fx.Flow('from_A', bus='Heat_A', size=200),  # Input from Site A
        out1=fx.Flow('to_B', bus='Heat_B', size=200),  # Output to Site B
        relative_losses=0.05,  # 5% heat loss in pipe
    ),
    # === Demands ===
    fx.Sink('Demand_A', inputs=[fx.Flow('Heat', bus='Heat_A', size=1, fixed_relative_profile=demand_a)]),
    fx.Sink('Demand_B', inputs=[fx.Flow('Heat', bus='Heat_B', size=1, fixed_relative_profile=demand_b)]),
)

fs_unidirectional.optimize(fx.solvers.HighsSolver());

In [5]:
# View results
print(f'Total cost: {fs_unidirectional.solution["costs"].item():.2f} €')

Total cost: 2479.00 €


In [6]:
# Heat balance at Site A
fs_unidirectional.statistics.plot.balance('Heat_A')

PlotResult(data=<xarray.Dataset> Size: 5kB
Dimensions:              (time: 169)
Coordinates:
  * time                 (time) datetime64[ns] 1kB 2024-01-22 ... 2024-01-29
Data variables:
    GasBoiler_A(Heat)    (time) float64 1kB -234.2 -242.0 -249.2 ... -192.8 nan
    Pipe_A_to_B(from_A)  (time) float64 1kB 84.21 84.21 84.21 ... 50.53 nan
    Demand_A(Heat)       (time) float64 1kB 150.0 157.8 165.0 ... 142.2 nan, figure=Figure({
    'data': [{'fillcolor': '#00CC96',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f} kW<extra></extra>',
              'legendgroup': 'GasBoiler_A(Heat)',
              'line': {'color': '#00CC96', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'GasBoiler_A(Heat)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'negative',
              'type': 'scatter',
              'x': array(['2024-01-22T00:00:00.000000000', '2024-01-22T01:00:00.000000000',
                          '2024-01-22T02:00:00.000000000', '2024-01-22T03:00:00.000000000',
                          '2024-01-22T04:00:00.000000000', '2024-01-22T05:00:00.000000000',
                          '2024-01-22T06:00:00.000000000', '2024-01-22T07:00:00.000000000',
                          '2024-01-22T08:00:00.000000000', '2024-01-22T09:00:00.000000000',
                          '2024-01-22T10:00:00.000000000', '2024-01-22T11:00:00.000000000',
                          '2024-01-22T12:00:00.000000000', '2024-01-22T13:00:00.000000000',
                          '2024-01-22T14:00:00.000000000', '2024-01-22T15:00:00.000000000',
                          '2024-01-22T16:00:00.000000000', '2024-01-22T17:00:00.000000000',
                          '2024-01-22T18:00:00.000000000', '2024-01-22T19:00:00.000000000',
                          '2024-01-22T20:00:00.000000000', '2024-01-22T21:00:00.000000000',
                          '2024-01-22T22:00:00.000000000', '2024-01-22T23:00:00.000000000',
                          '2024-01-23T00:00:00.000000000', '2024-01-23T01:00:00.000000000',
                          '2024-01-23T02:00:00.000000000', '2024-01-23T03:00:00.000000000',
                          '2024-01-23T04:00:00.000000000', '2024-01-23T05:00:00.000000000',
                          '2024-01-23T06:00:00.000000000', '2024-01-23T07:00:00.000000000',
                          '2024-01-23T08:00:00.000000000', '2024-01-23T09:00:00.000000000',
                          '2024-01-23T10:00:00.000000000', '2024-01-23T11:00:00.000000000',
                          '2024-01-23T12:00:00.000000000', '2024-01-23T13:00:00.000000000',
                          '2024-01-23T14:00:00.000000000', '2024-01-23T15:00:00.000000000',
                          '2024-01-23T16:00:00.000000000', '2024-01-23T17:00:00.000000000',
                          '2024-01-23T18:00:00.000000000', '2024-01-23T19:00:00.000000000',
                          '2024-01-23T20:00:00.000000000', '2024-01-23T21:00:00.000000000',
                          '2024-01-23T22:00:00.000000000', '2024-01-23T23:00:00.000000000',
                          '2024-01-24T00:00:00.000000000', '2024-01-24T01:00:00.000000000',
                          '2024-01-24T02:00:00.000000000', '2024-01-24T03:00:00.000000000',
                          '2024-01-24T04:00:00.000000000', '2024-01-24T05:00:00.000000000',
                          '2024-01-24T06:00:00.000000000', '2024-01-24T07:00:00.000000000',
                          '2024-01-24T08:00:00.000000000', '2024-01-24T09:00:00.000000000',
                          '2024-01-24T10:00:00.000000000', '2024-01-24T11:00:00.000000000',
                          '2024-01-24T12:00:00.000000000', '2024-01-24T13:00:00.000000000',
                          '2024-01-24T14:00:00.000000000', '2024-01-24T15:00:00.000000000',
                          '2024-01-24T16:00:00.000000000', '2024-01-24T

In [7]:
# Heat balance at Site B
fs_unidirectional.statistics.plot.balance('Heat_B')

PlotResult(data=<xarray.Dataset> Size: 5kB
Dimensions:             (time: 169)
Coordinates:
  * time                (time) datetime64[ns] 1kB 2024-01-22 ... 2024-01-29
Data variables:
    ElecBoiler_B(Heat)  (time) float64 1kB -0.0 -0.0 -0.0 -0.0 ... -0.0 -0.0 nan
    Pipe_A_to_B(to_B)   (time) float64 1kB -80.0 -80.0 -80.0 ... -48.0 -48.0 nan
    Demand_B(Heat)      (time) float64 1kB 80.0 80.0 80.0 80.0 ... 48.0 48.0 nan, figure=Figure({
    'data': [{'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f} kW<extra></extra>',
              'legendgroup': 'ElecBoiler_B(Heat)',
              'line': {'color': '#AB63FA', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'ElecBoiler_B(Heat)',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2024-01-22T00:00:00.000000000', '2024-01-22T01:00:00.000000000',
                          '2024-01-22T02:00:00.000000000', '2024-01-22T03:00:00.000000000',
                          '2024-01-22T04:00:00.000000000', '2024-01-22T05:00:00.000000000',
                          '2024-01-22T06:00:00.000000000', '2024-01-22T07:00:00.000000000',
                          '2024-01-22T08:00:00.000000000', '2024-01-22T09:00:00.000000000',
                          '2024-01-22T10:00:00.000000000', '2024-01-22T11:00:00.000000000',
                          '2024-01-22T12:00:00.000000000', '2024-01-22T13:00:00.000000000',
                          '2024-01-22T14:00:00.000000000', '2024-01-22T15:00:00.000000000',
                          '2024-01-22T16:00:00.000000000', '2024-01-22T17:00:00.000000000',
                          '2024-01-22T18:00:00.000000000', '2024-01-22T19:00:00.000000000',
                          '2024-01-22T20:00:00.000000000', '2024-01-22T21:00:00.000000000',
                          '2024-01-22T22:00:00.000000000', '2024-01-22T23:00:00.000000000',
                          '2024-01-23T00:00:00.000000000', '2024-01-23T01:00:00.000000000',
                          '2024-01-23T02:00:00.000000000', '2024-01-23T03:00:00.000000000',
                          '2024-01-23T04:00:00.000000000', '2024-01-23T05:00:00.000000000',
                          '2024-01-23T06:00:00.000000000', '2024-01-23T07:00:00.000000000',
                          '2024-01-23T08:00:00.000000000', '2024-01-23T09:00:00.000000000',
                          '2024-01-23T10:00:00.000000000', '2024-01-23T11:00:00.000000000',
                          '2024-01-23T12:00:00.000000000', '2024-01-23T13:00:00.000000000',
                          '2024-01-23T14:00:00.000000000', '2024-01-23T15:00:00.000000000',
                          '2024-01-23T16:00:00.000000000', '2024-01-23T17:00:00.000000000',
                          '2024-01-23T18:00:00.000000000', '2024-01-23T19:00:00.000000000',
                          '2024-01-23T20:00:00.000000000', '2024-01-23T21:00:00.000000000',
                          '2024-01-23T22:00:00.000000000', '2024-01-23T23:00:00.000000000',
                          '2024-01-24T00:00:00.000000000', '2024-01-24T01:00:00.000000000',
                          '2024-01-24T02:00:00.000000000', '2024-01-24T03:00:00.000000000',
                          '2024-01-24T04:00:00.000000000', '2024-01-24T05:00:00.000000000',
                          '2024-01-24T06:00:00.000000000', '2024-01-24T07:00:00.000000000',
                          '2024-01-24T08:00:00.000000000', '2024-01-24T09:00:00.000000000',
                          '2024-01-24T10:00:00.000000000', '2024-01-24T11:00:00.000000000',
                          '2024-01-24T12:00:00.000000000', '2024-01-24T13:00:00.000000000',
                          '2024-01-24T14:00:00.000000000', '2024-01-24T15:00:00.000000000',
                          '2024-01-24T16:00:00.000000000', '2024-01-24T17:00:00.000000000',
                          '2024-01-24T18:00:00

In [8]:
# Energy flow overview
fs_unidirectional.statistics.plot.sankey.flows()

PlotResult(data=<xarray.Dataset> Size: 1kB
Dimensions:  (link: 7)
Coordinates:
  * link     (link) int64 56B 0 1 2 3 4 5 6
    source   (link) <U11 308B 'Heat_A' 'Heat_B' 'Gas' ... 'Heat_A' 'Pipe_A_to_B'
    target   (link) <U11 308B 'Demand_A' 'Demand_B' ... 'Pipe_A_to_B' 'Heat_B'
    label    (link) <U19 532B 'Demand_A(Heat)' ... 'Pipe_A_to_B(to_B)'
    carrier  (link) <U4 112B 'heat' 'heat' 'gas' 'heat' 'gas' 'heat' 'heat'
Data variables:
    value    (link) float64 56B 2.52e+04 1.872e+04 ... 1.971e+04 1.872e+04, figure=Figure({
    'data': [{'link': {'color': [rgba(231, 76, 60, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(52, 152, 219, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(52, 152, 219, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(231, 76, 60, 0.4)],
                       'label': [Demand_A(Heat), Demand_B(Heat), GasBoiler_A(Gas),
                                 GasBoiler_A(Heat), GasSupply(Gas),
                                 Pipe_A_to_B(from_A), Pipe_A_to_B(to_B)],
                       'source': [5, 0, 4, 6, 1, 5, 7],
                       'target': [3, 2, 6, 5, 4, 7, 0],
                       'value': [25200.0, 18724.0, 41316.71578947369,
                                 44909.473684210534, 41316.71578947369,
                                 19709.473684210527, 18724.0]},
              'node': {'color': [#e74c3c, #636EFA, #FF6692, #19D3F3, #3498db,
                                 #e74c3c, #00CC96, #FFA15A],
                       'label': [Heat_B, GasSupply, Demand_B, Demand_A, Gas,
                                 Heat_A, GasBoiler_A, Pipe_A_to_B],
                       'line': {'color': 'black', 'width': 0.5},
                       'pad': 15,
                       'thickness': 20},
              'type': 'sankey'}],
    'layout': {'template': '...', 'title': {'text': 'Energy Flow'}}
}))

### Observations

- The optimizer uses the **cheaper gas boiler at Site A** as much as possible
- Heat is transmitted to Site B (despite 5% losses) because gas is much cheaper than electricity
- The electric boiler at Site B only runs when transmission capacity is insufficient

## Example 2: Bidirectional Transmission

Now allow heat to flow in **both directions**. This is useful when:
- Both sites have generation capacity
- Demand patterns differ between sites
- Prices or availability vary over time

In [9]:
# Add a heat pump at Site B (cheaper during certain hours)
# Electricity price varies: cheap at night, expensive during day
elec_price = np.where(
    (hour_of_day >= 22) | (hour_of_day <= 6),
    0.08,  # Night: 0.08 €/kWh
    0.25,  # Day: 0.25 €/kWh
)

In [10]:
fs_bidirectional = fx.FlowSystem(timesteps)
fs_bidirectional.add_carriers(
    fx.Carrier('gas', '#3498db', 'kW'),
    fx.Carrier('electricity', '#f1c40f', 'kW'),
    fx.Carrier('heat', '#e74c3c', 'kW'),
)
fs_bidirectional.add_elements(
    # === Buses ===
    fx.Bus('Heat_A', carrier='heat'),
    fx.Bus('Heat_B', carrier='heat'),
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Electricity', carrier='electricity'),
    # === Effect ===
    fx.Effect('costs', '€', 'Operating Costs', is_standard=True, is_objective=True),
    # === External supplies ===
    fx.Source('GasSupply', outputs=[fx.Flow('Gas', bus='Gas', size=1000, effects_per_flow_hour=0.06)]),
    fx.Source('ElecGrid', outputs=[fx.Flow('Elec', bus='Electricity', size=500, effects_per_flow_hour=elec_price)]),
    # === Site A: Gas boiler ===
    fx.LinearConverter(
        'GasBoiler_A',
        inputs=[fx.Flow('Gas', bus='Gas', size=500)],
        outputs=[fx.Flow('Heat', bus='Heat_A', size=400)],
        conversion_factors=[{'Gas': 1, 'Heat': 0.92}],
    ),
    # === Site B: Heat pump (efficient with variable electricity price) ===
    fx.LinearConverter(
        'HeatPump_B',
        inputs=[fx.Flow('Elec', bus='Electricity', size=100)],
        outputs=[fx.Flow('Heat', bus='Heat_B', size=350)],
        conversion_factors=[{'Elec': 1, 'Heat': 3.5}],  # COP = 3.5
    ),
    # === BIDIRECTIONAL Transmission ===
    fx.Transmission(
        'Pipe_AB',
        # Direction 1: A → B
        in1=fx.Flow('from_A', bus='Heat_A', size=200),
        out1=fx.Flow('to_B', bus='Heat_B', size=200),
        # Direction 2: B → A
        in2=fx.Flow('from_B', bus='Heat_B', size=200),
        out2=fx.Flow('to_A', bus='Heat_A', size=200),
        relative_losses=0.05,
        prevent_simultaneous_flows_in_both_directions=True,  # Can't flow both ways at once
    ),
    # === Demands ===
    fx.Sink('Demand_A', inputs=[fx.Flow('Heat', bus='Heat_A', size=1, fixed_relative_profile=demand_a)]),
    fx.Sink('Demand_B', inputs=[fx.Flow('Heat', bus='Heat_B', size=1, fixed_relative_profile=demand_b)]),
)

fs_bidirectional.optimize(fx.solvers.HighsSolver());

In [11]:
# Compare costs
print(f'Unidirectional cost: {fs_unidirectional.solution["costs"].item():.2f} €')
print(f'Bidirectional cost:  {fs_bidirectional.solution["costs"].item():.2f} €')
savings = fs_unidirectional.solution['costs'].item() - fs_bidirectional.solution['costs'].item()
print(f'Savings from bidirectional: {savings:.2f} €')

Unidirectional cost: 2479.00 €
Bidirectional cost:  2479.00 €
Savings from bidirectional: 0.00 €


In [12]:
# Visualize transmission flows in both directions using xarray
flow_data = xr.Dataset(
    {
        'A_to_B': fs_bidirectional.solution['Pipe_AB(from_A)|flow_rate'],
        'B_to_A': fs_bidirectional.solution['Pipe_AB(from_B)|flow_rate'],
    }
)

fig = px.line(
    x=list(flow_data['time'].values) * 2,
    y=np.concatenate([flow_data['A_to_B'].values, flow_data['B_to_A'].values]),
    color=['A → B'] * len(flow_data['time']) + ['B → A'] * len(flow_data['time']),
    title='Transmission Flow Direction Over Time',
    labels={'x': 'Time', 'y': 'Flow Rate [kW]', 'color': 'Direction'},
)
fig

In [13]:
# Heat balance at Site B showing bidirectional flows
fs_bidirectional.statistics.plot.balance('Heat_B')

PlotResult(data=<xarray.Dataset> Size: 7kB
Dimensions:           (time: 169)
Coordinates:
  * time              (time) datetime64[ns] 1kB 2024-01-22 ... 2024-01-29
Data variables:
    HeatPump_B(Heat)  (time) float64 1kB -0.0 -0.0 -0.0 -0.0 ... -0.0 -0.0 nan
    Pipe_AB(to_B)     (time) float64 1kB -80.0 -80.0 -80.0 ... -48.0 -48.0 nan
    Pipe_AB(from_B)   (time) float64 1kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 nan
    Demand_B(Heat)    (time) float64 1kB 80.0 80.0 80.0 80.0 ... 48.0 48.0 nan, figure=Figure({
    'data': [{'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f} kW<extra></extra>',
              'legendgroup': 'HeatPump_B(Heat)',
              'line': {'color': '#AB63FA', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'HeatPump_B(Heat)',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2024-01-22T00:00:00.000000000', '2024-01-22T01:00:00.000000000',
                          '2024-01-22T02:00:00.000000000', '2024-01-22T03:00:00.000000000',
                          '2024-01-22T04:00:00.000000000', '2024-01-22T05:00:00.000000000',
                          '2024-01-22T06:00:00.000000000', '2024-01-22T07:00:00.000000000',
                          '2024-01-22T08:00:00.000000000', '2024-01-22T09:00:00.000000000',
                          '2024-01-22T10:00:00.000000000', '2024-01-22T11:00:00.000000000',
                          '2024-01-22T12:00:00.000000000', '2024-01-22T13:00:00.000000000',
                          '2024-01-22T14:00:00.000000000', '2024-01-22T15:00:00.000000000',
                          '2024-01-22T16:00:00.000000000', '2024-01-22T17:00:00.000000000',
                          '2024-01-22T18:00:00.000000000', '2024-01-22T19:00:00.000000000',
                          '2024-01-22T20:00:00.000000000', '2024-01-22T21:00:00.000000000',
                          '2024-01-22T22:00:00.000000000', '2024-01-22T23:00:00.000000000',
                          '2024-01-23T00:00:00.000000000', '2024-01-23T01:00:00.000000000',
                          '2024-01-23T02:00:00.000000000', '2024-01-23T03:00:00.000000000',
                          '2024-01-23T04:00:00.000000000', '2024-01-23T05:00:00.000000000',
                          '2024-01-23T06:00:00.000000000', '2024-01-23T07:00:00.000000000',
                          '2024-01-23T08:00:00.000000000', '2024-01-23T09:00:00.000000000',
                          '2024-01-23T10:00:00.000000000', '2024-01-23T11:00:00.000000000',
                          '2024-01-23T12:00:00.000000000', '2024-01-23T13:00:00.000000000',
                          '2024-01-23T14:00:00.000000000', '2024-01-23T15:00:00.000000000',
                          '2024-01-23T16:00:00.000000000', '2024-01-23T17:00:00.000000000',
                          '2024-01-23T18:00:00.000000000', '2024-01-23T19:00:00.000000000',
                          '2024-01-23T20:00:00.000000000', '2024-01-23T21:00:00.000000000',
                          '2024-01-23T22:00:00.000000000', '2024-01-23T23:00:00.000000000',
                          '2024-01-24T00:00:00.000000000', '2024-01-24T01:00:00.000000000',
                          '2024-01-24T02:00:00.000000000', '2024-01-24T03:00:00.000000000',
                          '2024-01-24T04:00:00.000000000', '2024-01-24T05:00:00.000000000',
                          '2024-01-24T06:00:00.000000000', '2024-01-24T07:00:00.000000000',
                          '2024-01-24T08:00:00.000000000', '2024-01-24T09:00:00.000000000',
                          '2024-01-24T10:00:00.000000000', '2024-01-24T11:00:00.000000000',
                          '2024-01-24T12:00:00.000000000', '2024-01-24T13:00:00.000000000',
                          '2024-01-24T14:00:00.000000000', '2024-01-24T15:00:00.000000000',
                          '2024-01-24T16:00:00.000000000', '2024-01-24T

In [14]:
# Energy flow overview
fs_bidirectional.statistics.plot.sankey.flows()

PlotResult(data=<xarray.Dataset> Size: 1kB
Dimensions:  (link: 7)
Coordinates:
  * link     (link) int64 56B 0 1 2 3 4 5 6
    source   (link) <U11 308B 'Heat_A' 'Heat_B' 'Gas' ... 'Heat_A' 'Pipe_AB'
    target   (link) <U11 308B 'Demand_A' 'Demand_B' ... 'Pipe_AB' 'Heat_B'
    label    (link) <U17 476B 'Demand_A(Heat)' ... 'Pipe_AB(to_B)'
    carrier  (link) <U4 112B 'heat' 'heat' 'gas' 'heat' 'gas' 'heat' 'heat'
Data variables:
    value    (link) float64 56B 2.52e+04 1.872e+04 ... 1.971e+04 1.872e+04, figure=Figure({
    'data': [{'link': {'color': [rgba(231, 76, 60, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(52, 152, 219, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(52, 152, 219, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(231, 76, 60, 0.4)],
                       'label': [Demand_A(Heat), Demand_B(Heat), GasBoiler_A(Gas),
                                 GasBoiler_A(Heat), GasSupply(Gas),
                                 Pipe_AB(from_A), Pipe_AB(to_B)],
                       'source': [6, 0, 5, 7, 1, 6, 3],
                       'target': [4, 2, 7, 6, 5, 3, 0],
                       'value': [25200.0, 18724.0, 41316.71578947369,
                                 44909.473684210534, 41316.71578947369,
                                 19709.473684210527, 18724.0]},
              'node': {'color': [#e74c3c, #636EFA, #FF6692, #FFA15A, #19D3F3,
                                 #3498db, #e74c3c, #00CC96],
                       'label': [Heat_B, GasSupply, Demand_B, Pipe_AB, Demand_A,
                                 Gas, Heat_A, GasBoiler_A],
                       'line': {'color': 'black', 'width': 0.5},
                       'pad': 15,
                       'thickness': 20},
              'type': 'sankey'}],
    'layout': {'template': '...', 'title': {'text': 'Energy Flow'}}
}))

### Observations

- During **cheap electricity hours** (night): Heat pump at Site B produces heat, some flows to Site A
- During **expensive electricity hours** (day): Gas boiler at Site A supplies both sites
- The bidirectional transmission enables **load shifting** and **arbitrage** between sites

## Example 3: Transmission Capacity Optimization

What's the **optimal pipe capacity**? Let the optimizer decide.

In [15]:
# Daily amortized pipe cost (simplified)
PIPE_COST_PER_KW = 0.05  # €/kW/day capacity cost

fs_invest = fx.FlowSystem(timesteps)
fs_invest.add_carriers(
    fx.Carrier('gas', '#3498db', 'kW'),
    fx.Carrier('electricity', '#f1c40f', 'kW'),
    fx.Carrier('heat', '#e74c3c', 'kW'),
)
fs_invest.add_elements(
    # === Buses ===
    fx.Bus('Heat_A', carrier='heat'),
    fx.Bus('Heat_B', carrier='heat'),
    fx.Bus('Gas', carrier='gas'),
    fx.Bus('Electricity', carrier='electricity'),
    # === Effect ===
    fx.Effect('costs', '€', 'Operating Costs', is_standard=True, is_objective=True),
    # === External supplies ===
    fx.Source('GasSupply', outputs=[fx.Flow('Gas', bus='Gas', size=1000, effects_per_flow_hour=0.06)]),
    fx.Source('ElecGrid', outputs=[fx.Flow('Elec', bus='Electricity', size=500, effects_per_flow_hour=elec_price)]),
    # === Site A: Gas boiler ===
    fx.LinearConverter(
        'GasBoiler_A',
        inputs=[fx.Flow('Gas', bus='Gas', size=500)],
        outputs=[fx.Flow('Heat', bus='Heat_A', size=400)],
        conversion_factors=[{'Gas': 1, 'Heat': 0.92}],
    ),
    # === Site B: Heat pump ===
    fx.LinearConverter(
        'HeatPump_B',
        inputs=[fx.Flow('Elec', bus='Electricity', size=100)],
        outputs=[fx.Flow('Heat', bus='Heat_B', size=350)],
        conversion_factors=[{'Elec': 1, 'Heat': 3.5}],
    ),
    # === Site B: Backup electric boiler ===
    fx.LinearConverter(
        'ElecBoiler_B',
        inputs=[fx.Flow('Elec', bus='Electricity', size=200)],
        outputs=[fx.Flow('Heat', bus='Heat_B', size=200)],
        conversion_factors=[{'Elec': 1, 'Heat': 0.99}],
    ),
    # === Transmission with INVESTMENT OPTIMIZATION ===
    # Investment parameters are passed via 'size' parameter
    fx.Transmission(
        'Pipe_AB',
        in1=fx.Flow(
            'from_A',
            bus='Heat_A',
            size=fx.InvestParameters(
                effects_of_investment_per_size={'costs': PIPE_COST_PER_KW * 7},  # Weekly cost
                minimum_size=0,
                maximum_size=300,
            ),
        ),
        out1=fx.Flow('to_B', bus='Heat_B'),
        in2=fx.Flow(
            'from_B',
            bus='Heat_B',
            size=fx.InvestParameters(
                effects_of_investment_per_size={'costs': PIPE_COST_PER_KW * 7},
                minimum_size=0,
                maximum_size=300,
            ),
        ),
        out2=fx.Flow('to_A', bus='Heat_A'),
        relative_losses=0.05,
        balanced=True,  # Same capacity in both directions
        prevent_simultaneous_flows_in_both_directions=True,
    ),
    # === Demands ===
    fx.Sink('Demand_A', inputs=[fx.Flow('Heat', bus='Heat_A', size=1, fixed_relative_profile=demand_a)]),
    fx.Sink('Demand_B', inputs=[fx.Flow('Heat', bus='Heat_B', size=1, fixed_relative_profile=demand_b)]),
)

fs_invest.optimize(fx.solvers.HighsSolver());

In [16]:
# Results
optimal_capacity = fs_invest.solution['Pipe_AB(from_A)|size'].item()
total_cost = fs_invest.solution['costs'].item()

print(f'Optimal pipe capacity: {optimal_capacity:.1f} kW')
print(f'Total cost: {total_cost:.2f} €')

Optimal pipe capacity: 189.5 kW
Total cost: 2611.63 €


In [17]:
# Effect breakdown by component
fs_invest.statistics.plot.effects()

PlotResult(data=<xarray.Dataset> Size: 16B
Dimensions:  ()
Data variables:
    costs    float64 8B 2.612e+03
    Penalty  float64 8B 0.0, figure=Figure({
    'data': [{'hovertemplate': 'variable=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'costs',
              'marker': {'color': '#636EFA', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'costs',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['costs'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'iR2i4ERnpEA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Penalty',
              'marker': {'color': '#EF553B', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'Penalty',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Penalty'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAAAA=', 'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'bargap': 0,
               'bargroupgap': 0,
               'barmode': 'relative',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Effects (total)'},
               'xaxis': {'anchor': 'y',
                         'categoryarray': [costs, Penalty],
                         'categoryorder': 'array',
                         'domain': [0.0, 1.0],
                         'title': {'text': 'variable'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
}))

In [18]:
# Energy flows
fs_invest.statistics.plot.sankey.flows()

PlotResult(data=<xarray.Dataset> Size: 1kB
Dimensions:  (link: 7)
Coordinates:
  * link     (link) int64 56B 0 1 2 3 4 5 6
    source   (link) <U11 308B 'Heat_A' 'Heat_B' 'Gas' ... 'Heat_A' 'Pipe_AB'
    target   (link) <U11 308B 'Demand_A' 'Demand_B' ... 'Pipe_AB' 'Heat_B'
    label    (link) <U17 476B 'Demand_A(Heat)' ... 'Pipe_AB(to_B)'
    carrier  (link) <U4 112B 'heat' 'heat' 'gas' 'heat' 'gas' 'heat' 'heat'
Data variables:
    value    (link) float64 56B 2.52e+04 1.872e+04 ... 1.971e+04 1.872e+04, figure=Figure({
    'data': [{'link': {'color': [rgba(231, 76, 60, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(52, 152, 219, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(52, 152, 219, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(231, 76, 60, 0.4)],
                       'label': [Demand_A(Heat), Demand_B(Heat), GasBoiler_A(Gas),
                                 GasBoiler_A(Heat), GasSupply(Gas),
                                 Pipe_AB(from_A), Pipe_AB(to_B)],
                       'source': [6, 0, 5, 7, 1, 6, 3],
                       'target': [4, 2, 7, 6, 5, 3, 0],
                       'value': [25200.0, 18724.0, 41316.71578947369,
                                 44909.473684210534, 41316.71578947369,
                                 19709.47368421051, 18723.99999999998]},
              'node': {'color': [#e74c3c, #636EFA, #B6E880, #19D3F3, #FF6692,
                                 #3498db, #e74c3c, #00CC96],
                       'label': [Heat_B, GasSupply, Demand_B, Pipe_AB, Demand_A,
                                 Gas, Heat_A, GasBoiler_A],
                       'line': {'color': 'black', 'width': 0.5},
                       'pad': 15,
                       'thickness': 20},
              'type': 'sankey'}],
    'layout': {'template': '...', 'title': {'text': 'Energy Flow'}}
}))

## Key Concepts

### Transmission Component Structure

```python
fx.Transmission(
    label='pipe_name',
    # Direction 1: A → B
    in1=fx.Flow('from_A', bus='Bus_A', size=100),
    out1=fx.Flow('to_B', bus='Bus_B', size=100),
    # Direction 2: B → A (optional - omit for unidirectional)
    in2=fx.Flow('from_B', bus='Bus_B', size=100),
    out2=fx.Flow('to_A', bus='Bus_A', size=100),
    # Loss parameters
    relative_losses=0.05,  # 5% proportional loss
    absolute_losses=10,    # 10 kW fixed loss when active (optional)
    # Operational constraints
    prevent_simultaneous_flows_in_both_directions=True,
    balanced=True,  # Same capacity both directions (needs InvestParameters)
)
```

### Loss Types

| Loss Type | Formula | Use Case |
|-----------|---------|----------|
| **Relative** | `out = in × (1 - loss)` | Heat pipes, electrical lines |
| **Absolute** | `out = in - loss` (when active) | Pump energy, standby losses |

### Bidirectional vs Unidirectional

| Configuration | Parameters | Use Case |
|---------------|------------|----------|
| **Unidirectional** | `in1`, `out1` only | One-way pipelines, conveyors |
| **Bidirectional** | `in1`, `out1`, `in2`, `out2` | Power lines, reversible pipes |

### Investment Optimization

Use `InvestParameters` as the `size` parameter for capacity optimization:

```python
in1=fx.Flow(
    'from_A', 
    bus='Bus_A',
    size=fx.InvestParameters(  # Pass InvestParameters as size
        effects_of_investment_per_size={'costs': cost_per_kw},
        minimum_size=0,
        maximum_size=500,
    ),
)
```

## Common Use Cases

| Application | Typical Losses | Notes |
|-------------|---------------|-------|
| **District heating pipe** | 2-10% relative | Temperature-dependent |
| **High voltage line** | 1-5% relative | Distance-dependent |
| **Natural gas pipeline** | 0.5-2% relative | Compressor energy as absolute loss |
| **Conveyor belt** | Fixed absolute | Motor energy consumption |
| **Hydrogen pipeline** | 1-3% relative | Compression losses |

## Summary

You learned how to:

- Create **unidirectional transmission** between two buses
- Model **bidirectional transmission** with flow direction constraints
- Apply **relative and absolute losses** to transmission
- Optimize **transmission capacity** using InvestParameters
- Analyze **multi-site energy systems** with interconnections

### Next Steps

- **[07-scenarios-and-periods](07-scenarios-and-periods.ipynb)**: Multi-year planning with uncertainty
- **[08a-Aggregation](08a-aggregation.ipynb)**: Speed up large problems with time series aggregation
- **[08b-Rolling Horizon](08b-rolling-horizon.ipynb)**: Decompose large problems into sequential segments